In [2]:
# first convert ods to csv

import pandas as pd
import pickle
from aksharamukha import transliterate
import re

read in inflections table

In [2]:
print("~" * 40)
print("inflection generator.ipynb")

inflection_df = pd.read_excel('/home/bhikkhu/Bodhirasa/Dropbox/dpd/inflection-generator/declensions & conjugations.xlsx', sheet_name="declensions", dtype=str)

inflection_df = inflection_df.shift(periods=2)

inflection_df.columns = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", "AA", "AB", "AC", "AD", "AE", "AF", "AG", "AH", "AI", "AJ", "AK", "AL", "AM", "AN", "AO", "AP", "AQ", "AR", "AS", "AT", "AU", "AV", "AW", "AX", "AY", "AZ", "BA", "BB", "BC", "BD", "BE", "BF", "BG", "BH", "BI", "BJ", "BK", "BL", "BM", "BN", "BO", "BP", "BQ", "BR", "BS", "BT", "BU", "BV", "BW", "BX", "BY", "BZ", "CA", "CB", "CC", "CD", "CE", "CF", "CG", "CH", "CI", "CJ", "CK", "CL", "CM", "CN", "CO", "CP", "CQ", "CR", "CS", "CT", "CU", "CV", "CW", "CX", "CY", "CZ", "DA", "DB", "DC", "DD", "DE", "DF", "DG", "DH", "DI", "DJ", "DK"]
inflection_df.fillna("", inplace=True)

read index

In [3]:
index_df = pd.read_excel('/home/bhikkhu/Bodhirasa/Dropbox/dpd/inflection-generator/declensions & conjugations.xlsx', sheet_name="index", dtype=str)
index_df.fillna("", inplace=True)
# index_df["inflection name"] = index_df["inflection name"].str.replace(" ", "_") #only needed if using as variable
index_df_length = len(index_df)

index_df.columns
index_dict = dict(zip(index_df.iloc[:, 0], index_df.iloc[:, 2]))

generate df's of each inflection pattern

In [4]:
import re

for row in range(index_df_length):
	inflection_name = index_df.iloc[row,0]
	cell_range = index_df.iloc[row,1]
	like = index_df.iloc[row,2]
	irreg = index_df.iloc[row,3]

	col_range_1 = re.sub("(.+?)\d*\:.+", "\\1", cell_range)
	col_range_2 = re.sub(".+\:(.[A-Z]*)\d*", "\\1", cell_range)
	row_range_1 = int(re.sub(".+?(\d{1,3}):.+", "\\1", cell_range))
	row_range_2 = int(re.sub(".+:.+?(\d{1,3})", "\\1", cell_range))

	inflection_df_filtered = inflection_df.loc[row_range_1:row_range_2, col_range_1:col_range_2]
	inflection_df_filtered.Name =  f"{inflection_name}" #help! change the name to "inflection name" 

	inflection_df_filtered.iloc[0,0] = ""

	inflection_df_filtered.to_csv(f"output/patterns/{inflection_name}.csv", sep="\t", index=False, header=False)

create df and test

In [5]:
dpd_df = pd.read_csv("/home/bhikkhu/Bodhirasa/Dropbox/dpd/csvs/dpd-full.csv", sep="\t", dtype=str)
dpd_df.fillna("", inplace=True) 
dpd_df_length = len(dpd_df)

#empty stem

error = False

for row in range(dpd_df_length):
	headword = dpd_df.loc[row, "Pāli1"]
	stem = dpd_df.loc[row, "Stem"]
	pattern = dpd_df.loc[row, "Pattern"]
	
	if stem == "":
		print(f"stem error: {headword} has no stem!")
		error = True
	if stem != "-" and pattern == "":
		print(f"pattern error: {headword} has no pattern!")
		error = True

if error == True:
	input("there are stem & pattern errors, please fix them before continuiing")
	exit()
else:
	print("no stem & pattern errors found, continuing ...")


no stem & pattern errors found, continuing ...


generate all inflection tables & synonyms

In [6]:
print("~" * 40)
print("generating inflection tables & synonyms")
print("~" * 40)

indeclinables = ["abbrev", "abs", "ger", "ind", "inf", "prefix"]
conjugations = ["aor", "cond", "fut", "imp", "imperf", "opt", "perf", "pr"]
declensions = ["adj", "card", "cs", "fem", "letter", "masc", "nt", "ordin", "pp", "pron", "prp", "ptp", "root", "suffix", "ve"]

row = 0

all_synonyms = open ("output/all synonyms.csv", "w")

for row in range(dpd_df_length): #dpd_df_length
	headword = dpd_df.loc[row, "Pāli1"]
	headword_clean = re.sub(" \d*$", "", headword)
	stem = dpd_df.loc[row, "Stem"]
	if stem == "*":
		stem = ""
	pattern = dpd_df.loc[row, "Pattern"]
	pos = dpd_df.loc[row, "POS"]
	metadata = dpd_df.loc[row, "Metadata"]
	meaning = dpd_df.loc[row, "Meaning IN CONTEXT"]
	synonyms = ""

	if row % 5000 == 0:
		print(f"{row}/{dpd_df_length}\t{headword}")
	
	with open(f"output/html/{headword}.html", "w") as html_table:

		if stem == "-":
			html_table.write(f"<p><b>{headword_clean}</b> is indeclinable")
			synonyms += headword_clean + " "

		elif stem == "!":
			html_table.write(f"<p>click on <b>{pattern}</b> for inflection table")
			synonyms += headword_clean + " "

		else:
			df = pd.read_csv(f"output/patterns/{pattern}.csv", sep="\t", index_col=0)
			df.fillna("", inplace=True, axis=0)
			
			df_rows = df.shape[0]
			df_columns = df.shape[1]

			for rows in range(0, df_rows): 
				for columns in range(0, df_columns, 2): #1 to 0
					
					html_cell = df.iloc[rows, columns]
					syn_cell = df.iloc[rows, columns]	

					if html_cell == "sg" or html_cell == "pl":
						pass

					else:
						html_cell = re.sub(r"(.+)", f"<b>\\1</b>", html_cell) # add bold
						html_cell = re.sub(r"(.+)", f"{stem}\\1", html_cell) # add stem
						html_cell = re.sub(r"\n", "<br>", html_cell) # add line breaks
						df.iloc[rows, columns] = html_cell
						
						syn_cell = re.sub(r"(.+)", f"{stem}\\1", syn_cell)
						search_string = re.compile("\n", re.M)
						replace_string = " "
						matches = re.sub(search_string, replace_string, syn_cell)
						synonyms += matches + " "
			
			column_list = []
			for i in range(1, df_columns, 2):
				column_list.append(i)

			df.drop(df.columns[column_list], axis=1, inplace=True)
			table = df.to_html(escape=False)
			table = re.sub("Unnamed.+", "", table)
			table = re.sub("NaN", "", table)

			# write header info

			if index_dict[pattern] != "":
				if pos in declensions:
					heading = (f"""<p class="heading"><b>{headword_clean}</b> is <b>{pattern}</b> declension like <b>{index_dict[pattern]}</b></p>""")
				if pos in conjugations:
					heading = (f"""<p class="heading"><b>{headword_clean}</b> is <b>{pattern}</b> conjugation like <b>{index_dict[pattern]}</b></p>""")

			if index_dict[pattern] == "":
				if pos in declensions:
					heading = (f"""<p class="heading"><b>{headword_clean}</b> is <b>{pattern}</b> irregular declension</p>""")
				if pos in conjugations:
					heading = (f"""<p class="heading"><b>{headword_clean}</b> is <b>{pattern}</b> irregular conjugation</p>""")
			
			html = heading + table
			html_table.write(html)

	all_synonyms.write(f"{headword}	{synonyms}\n")

all_synonyms.close()

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
generating inflection tables
0 akakkasa
5000 ambila 2
10000 ubhayathā
15000 guṇa 3
20000 duggandha 1
25000 paṭisevi
30000 buddhālambana
35000 vajjetvā
40000 sandhiccheda 1


transcribe synonyms

In [8]:
print("~" * 40)
print("converting synonyms to sinhala and devanagari")


all_synonyms = open("output/all synonyms.csv", "r")
all_synonyms_read = all_synonyms.read()
all_synonyms.close()

all_synonyms_translit = open("output/all synonyms translt.csv", "w")

sinhala = transliterate.process("IAST","Sinhala", all_synonyms_read, post_options =['SinhalaPali', 'SinhalaConjuncts'])
devanagari = transliterate.process("IAST","Devanagari",all_synonyms_read, post_options = ['DevanagariAnusvara'])

roman = all_synonyms_read.split("\n")[:-1]
sinhala = sinhala.split("\n")
devanagari = devanagari.split("\n")

for i in zip(roman, sinhala, devanagari):	
  all_synonyms_translit.write(i[0]+i[1].split("\t")[1]+i[2].split("\t")[1]+"\n")

all_synonyms_translit.close()

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
converting synonyms to sinhala and devanagari
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


export to pickle

In [5]:
print("~" * 40)
print("exporting synonyms to pickle")


all_synonyms = pd.read_csv("output/all synonyms translt.csv", header=None, sep="\t")

length = len(all_synonyms)

for row in range(length):
	headword = all_synonyms.iloc[row, 0]
	synonyms = all_synonyms.iloc[row, 1]

	synonyms_list = synonyms.split()

	# add ṁ version

	for word in synonyms_list:
		if 'ṃ' in word:
			wordṁ = re.sub("ṃ", "ṁ", word) 
			synonyms_list.append(wordṁ)

	synonyms_list = list(dict.fromkeys(synonyms_list))

	with open(f"output/synonyms/{headword}", "wb") as text_file:
		pickle.dump(synonyms_list, text_file)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
exporting synonyms to pickle
